In [ ]:
import os, sys
sys.path.append('../../src')

import json
from datetime import datetime
import pandas as pd

import visualization.visualize as vis
from data.load_dataset import load_dataset
from models.train_model import get_dataloaders, set_and_train_model
from models.model import get_model, evaluate_model, save_model

## Running all databases and rounds

In [ ]:
num_epochs = 30
base_path = '../../models/local_models/'

In [ ]:
exec_time = datetime.now()
exec_id = f'{exec_time.year}-{exec_time.month}-{exec_time.day}_{exec_time.hour}h{exec_time.minute}m{exec_time.second}s'
base_path = os.path.join(base_path, f'{exec_id}/')
if not os.path.isdir(base_path):
    os.mkdir(base_path)
    print(base_path, 'folder created.')

model_predictions = []
# round_number_list = [1,2]
# site_number_list = [1,2]
round_number_list = [1,2,3,4,5]
site_number_list = [1,2,3,4,5]
for round_number in round_number_list:
    for site_number in site_number_list:

        print(f'\n\nTraining round_number {round_number} site_number {site_number}:')
        model_id = f'local_model_round-{round_number}_site-{site_number}'
        model_id_old = f'local_model_round-{round_number-1}_site-{site_number}'
        model_name = f'{exec_id}_{model_id}'
        model_name_old = f'{exec_id}_{model_id_old}'
        
        df = load_dataset(site_number)
        dataloaders, db_sizes = get_dataloaders(df, round_number, test_all_rounds=True)

        ### Visualize a few images
        # Let's visualize a few training images so as to understand the data augmentations.
        # vis.plot_samples(dataloaders['train'], title=f'site_number {site_number} round_number {round_number} samples')        
        
        # Train the model
        model = get_model(model_name=model_name_old, base_path=base_path)
        model, model_hist = set_and_train_model(model, dataloaders, num_epochs=num_epochs)
        vis.plot_model_hist(model_hist, f'Baseline Model site_number: {site_number} round_number: {round_number}')
        
        # Evaluate model predictions
        predictions = evaluate_model(model, dataloaders, split='test')
        predictions['round_number'] = round_number; predictions['site_number'] = site_number
        model_predictions.append(predictions)
        # Saving current model:
        save_model(model, model_name, base_path=base_path)

In [ ]:
# Saving predictions report:
json_filepath = f'{base_path}{exec_id}_predictions.json'
with open(json_filepath, 'w') as outfile:
    json.dump(model_predictions, outfile)

In [ ]:
with open(json_filepath) as json_file:
    data = json.load(json_file)
model_predictions_ = pd.DataFrame(data)
model_predictions_